In [1]:
#!pip uninstall tensorflow --yes

# this and the next cells to make sure we use latest releases

In [2]:
#!pip uninstall keras --yes


In [3]:
#!pip install tensorflow
#!pip install keras

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

2023-07-21 21:22:49.689743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 21:22:50.344292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model





In [6]:
# Sample data (replace this with your actual dataset)
#  100 data samples, each having 64 features. 
data = np.random.rand(100, 64)  # Assuming 100 data samples with 64 features each

# Step 1: Create and train the autoencoder
input_dim = data.shape[1]
# Sets input_dim to the number of features in the data: 64 
latent_dim = 32  # Adjust this according to your desired latent space size




In [7]:
# Encoder
input_data = Input(shape=(input_dim,))
#
# Defines the input layer of the autoencoder with the shape (input_dim,), 
# which accepts data samples with 64 features each.
#
encoded = Dense(latent_dim, activation='relu')(input_data)
#
#Defines the encoder part of the autoencoder. It takes the input data and
#applies a dense (fully connected) layer with latent_dim neurons
#and a ReLU activation function to compress the data into the latent space.
#
encoder = Model(input_data, encoded)
#Creates the encoder model using the Model class with the input as input_data and 
#the output as encoded. This forms the encoder part of the autoencoder.


2023-07-21 21:22:51.183484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-21 21:22:51.183549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: joepareti54-GS75-Stealth-9SE
2023-07-21 21:22:51.183556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: joepareti54-GS75-Stealth-9SE
2023-07-21 21:22:51.183674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.41.3
2023-07-21 21:22:51.183704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.41.3
2023-07-21 21:22:51.183709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.41.3


In [8]:
# Decoder
decoder_input = Input(shape=(latent_dim,))
# Defines the input layer of the decoder 
#with the shape (latent_dim,), 
#which accepts data samples with 32 features 
#(matching the size of the latent space).
#
decoded = Dense(input_dim, activation='sigmoid')(decoder_input)
#Defines the decoder part of the autoencoder. 
#It takes the compressed data from the encoder (decoder_input) 
#and applies a dense layer with input_dim neurons and 
#a hyperbolic tangent (tanh) activation function 
#to reconstruct the original data.
#
decoder = Model(decoder_input, decoded)
#Creates the decoder model using the Model class with the input 
#as decoder_input and the output as decoded.




In [9]:
# Autoencoder (combination of encoder and decoder)
autoencoder_input = Input(shape=(input_dim,))
# Defines the input layer for the autoencoder with the shape 
# (input_dim,), which accepts data samples with 64 features each.
autoencoder_encoded = encoder(autoencoder_input)
# Applies the encoder model (encoder) to the input data to get 
#the compressed representation in the latent space.
#
autoencoder_decoded = decoder(autoencoder_encoded)
# Applies the decoder model (decoder) to the compressed 
# representation to reconstruct the original data.
#
autoencoder = Model(autoencoder_input, autoencoder_decoded)
# Creates the complete autoencoder model using the 
# Model class with the input as autoencoder_input and 
# the output as autoencoder_decoded.
#
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64)]              0         
                                                                 
 model (Functional)          (None, 32)                2080      
                                                                 
 model_1 (Functional)        (None, 64)                2112      
                                                                 
Total params: 4192 (16.38 KB)
Trainable params: 4192 (16.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
autoencoder.fit(data, data, epochs=50, batch_size=32)
#Trains the autoencoder using the fit method with data 
#as both the input and output (since it is an autoencoder). 
# The autoencoder is trained for 50 epochs with a batch size of 32.

Epoch 1/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0930
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0893
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0870
Epoch 4/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0855
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0845
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0839
Epoch 7/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0834
Epoch 8/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0829
Epoch 9/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0826
Epoch 10/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0824
Epoch 11/50
4/4 [==============================] - 0s 1ms/step - loss: 0.0822
Epoch 12/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0820
Epoch 13/50
4/4 [==============================] - 0s 2ms/step - loss: 0.

In [11]:
# Step 2: Obtain the latent space representation
latent_representation = encoder.predict(data)
# After training the autoencoder, this step obtains the 
# compressed representation (latent space) of the input data 
# using the encoder model.

# Step 3: Apply Perturbation
epsilon = 0.1  # Adjust the perturbation strength as needed
epsilon = 0.0
perturbation = np.random.uniform(-epsilon, epsilon, size=latent_representation.shape)
perturbed_latent_representation = latent_representation + perturbation
# Applies the perturbation to the latent space representation 
# to create a perturbed latent space.
#
# Step 4: Decode Perturbed Latent Representations
perturbed_data = decoder.predict(perturbed_latent_representation)
# Decodes the perturbed latent space representation using 
# the decoder model to obtain the perturbed data. 
# The decoder maps the perturbed latent space back to the original data space.
#
# Step 5: Observe the Results
# Compare the perturbed_data with the original data to observe the impact of perturbations.
print("Original Data Shape:", data.shape)
print("Perturbed Data Shape:", perturbed_data.shape)
# The code then prints the shapes of the original data and 
# the perturbed data to observe the impact of the perturbations.

4/4 [==============================] - 0s 841us/step
Original Data Shape: (100, 64)
Perturbed Data Shape: (100, 64)


In [12]:
print (data)

[[0.05925131 0.32499617 0.79758326 ... 0.62563696 0.93915596 0.89899897]
 [0.40866064 0.90505149 0.48497801 ... 0.32777383 0.46564943 0.52200889]
 [0.3109939  0.26114969 0.7439169  ... 0.07993909 0.79106413 0.83646685]
 ...
 [0.76416097 0.65446779 0.16577043 ... 0.34411686 0.55202363 0.73164175]
 [0.39115989 0.95446798 0.10611404 ... 0.05872074 0.76033107 0.17118192]
 [0.98650528 0.64255723 0.8388376  ... 0.85932459 0.92463918 0.4511323 ]]


In [13]:
print(perturbed_data)

[[0.46602824 0.41471514 0.42414075 ... 0.50805926 0.7086074  0.57131517]
 [0.45813963 0.4943963  0.42384756 ... 0.5560524  0.64045507 0.5520639 ]
 [0.47869372 0.4495801  0.45897588 ... 0.4608359  0.47061217 0.541848  ]
 ...
 [0.47564748 0.59807605 0.44935754 ... 0.5187882  0.45933253 0.5661952 ]
 [0.44187322 0.6026424  0.44098148 ... 0.50940776 0.5067764  0.48579106]
 [0.47676736 0.43748862 0.6706972  ... 0.49686617 0.61880803 0.49802032]]
